In [86]:
import scipy.stats as stats
from scipy import interpolate
import numpy as np
import pandas as pd
import os
import math

In [87]:
######################## Input parameters #########################
files = 200
points = 100000
case = 'case_c'   #case_a, case_b, case_c

athm_pr = 100000                           # [Pa]
dm = 3.8E-10                               # [-]
R = 8.314462                               # [J/(mol - K)]
M = 1.6E-2                                 # [kg/mol]
Na = 6.0221415E23                          # [1/mol]
b = -1
dpdl = 0.1                                 # [MPa/m]

In [88]:
def wu(pars, f_z, f_vis, f_cg, f_rho):
    p = pars[0]                             # [Pa]
    ro = pars[1]                            # [m]
    por = pars[2]                           # [-]
    tor = pars[3]                           # [-]
    op = pars[4]                            # [Pa]
    q = pars[5]                             # [-]
    t = pars[6]                             # [-]
    k = pars[7]                             # [-]
    fd = pars[8]                            # [-]
    ih = pars[9]                            # [J/mol]
    lp0 = pars[10]                          # [Pa]
    a0 = pars[11]                           # [-]
    a1 = pars[12]                           # [-]
    beta = pars[13]                         # [-]
    T = pars[14]                            # [K]

    Z = interpolate.bisplev(p,T,f_z)
    vis = interpolate.bisplev(p,T,f_vis)
    rho = interpolate.bisplev(p,T,f_rho)
    cg = f_cg(p)
    dzdp = (interpolate.bisplev(p+10000,T,f_z)-interpolate.bisplev(p-10000,T,f_z))/20000
    
    porosity = por*((op-p)/athm_pr)**-q  
    radius = ro*((op-p)/athm_pr)**-t 
    kn = (vis/(p*2*radius))*(math.pi*Z*R*T/(2*M))**0.5
    wv = 1/(1+kn)
    wk = 1-wv
    lp = lp0 * math.exp(-ih/(R*T))
    theta = (p/Z)/(lp+p/Z)
    rad = theta*dm    ####AAAAAAAAAAAAAAAAAAAAAAAAA
    fc1 = (porosity/tor)*(1-rad/radius)**2
    fc2 = fc1 * (((1-rad/radius)**-2)-1)
    alpha = a0*2*math.atan(a1*kn**beta)/math.pi 
    B1 = wv*fc1*radius**2*p*M*(1+alpha*kn)*(1+4*kn/(1-b*kn))/(8*vis*Z*R*T)
    B2 = wk*fc1*2*radius*(dm/radius)**(fd-2)*((8*Z*M/(math.pi*R*T))**0.5)*p*cg/(3*Z)
    if k>=1:
        gam = 0
    else:
        gam = 1 
    ds0 = 8.29E-7 * T**0.5 * math.exp(-ih**0.8/(R*T)) # m2/s
    ds = ds0 * ((1-theta) + 0.5*k*theta*(2-theta) + gam*0.5*k*(1-k)*(theta**2))/((1-theta+0.5*k*theta)**2) # m2/s
    Csc = 4*theta*M/(math.pi*dm**3*Na) # kg*mol/mol*m^3    kg/m^3
    B3 = fc2*ds*Csc/(p) # m^2*kg*s^2*m^2/s*m^3*kg*m = s
    D1 = (B1*R/M)/(1/(T*Z)+p*dzdp/(T*Z**2))
    D2 = (B2*R/M)/(1/(T*Z)+p*dzdp/(T*Z**2))
    D3 = (B3*R/M)/(1/(T*Z)+p*dzdp/(T*Z**2))
    DT = D1+D2+D3
    J = (B1+B2+B3)*dpdl*86400*365/1000
    return (B1, B2, B3, J, D1, D2, D3, DT, kn, dzdp, Z, cg, vis, rho)

# J [Ton/m2/year]
# vis [Pa*s]

In [89]:
########### Be sure where to start #############
existing_files = 0
for file in os.listdir('../Results/model_eval/'+case+'/.'):
    if file.startswith('wu_eval_'):
        existing_files += 1

In [90]:
########### Interpolate the state equation #############
properties = pd.read_csv('../z,cg,n.csv', delimiter=';')
p = properties['P(MPa)'].values*1000000
t = properties['T(K)'].values
f_z = interpolate.bisplrep(p, t, properties['Z'].values, s=2)
f_rho = interpolate.bisplrep(p, t, properties['Density(kg/m3)'].values, s=2)
f_vis = interpolate.bisplrep(p, t, properties['Viscosity(uPa-s)'].values/1000000, s=2)
f_cg = interpolate.interp1d(p, properties['cg(1/Mpa)'].values/1000000)

In [91]:
########### Create variability ranges #############
var_ranges = pd.read_csv('../variability_ranges.csv').set_index('Variable')
lower_lim = var_ranges['Min'].values
upper_lim = var_ranges['Max'].values
mean_val = (var_ranges['Max'].values + var_ranges['Min'].values)/2
std_val = (var_ranges['Max'].values - var_ranges['Min'].values)/math.sqrt(12)

In [92]:
# import matplotlib.pyplot as plt

# rand_radius_uniform = np.random.uniform(low=lower_lim[1], high=upper_lim[1], size=(points))
# rand_radius_normal = stats.truncnorm((lower_lim[1]-mean_val[1])/(std_val[1]), (upper_lim[1]-mean_val[1])/(std_val[1]), loc=mean_val[1], scale=(std_val[1])).rvs((points))

# rand_radius_ln_old = np.random.lognormal(mean = -18.165, sigma = 1.01, size = int(points*1.4))
# rand_radius_ln_new = stats.lognorm.rvs(math.sqrt(math.log(1 + ((std_val[1])/(mean_val[1]))**2)), 0, (mean_val[1])/math.sqrt(1 + ((std_val[1])/(mean_val[1]))**2),  int(points*1.4))
# rand_radius_trunc_old = rand_radius_ln_old[np.where((rand_radius_ln_old >= lower_lim[1]) & (rand_radius_ln_old < upper_lim[1]))][:points]
# rand_radius_trunc_new = rand_radius_ln_new[np.where((rand_radius_ln_new >= lower_lim[1]) & (rand_radius_ln_new < upper_lim[1]))][:points]

# scale = np.logspace(-10,-6,100)
# scale = np.linspace(1E-9,1E-7,100)
# plt.hist(rand_radius_trunc_new, bins=scale, density=True, label = 'truncated_new', histtype = 'step')
# plt.hist(rand_radius_uniform, bins=scale, density=True, label = 'uniform', histtype = 'step')
# plt.hist(rand_radius_normal, bins=scale, density=True, label = 'normal', histtype = 'step')
# # plt.xscale('log')
# plt.legend()

# print('uniform')
# print(np.mean(rand_radius_uniform))
# print(np.std(rand_radius_uniform))

# print('normal')
# print(np.mean(rand_radius_normal))
# print(np.std(rand_radius_normal))

# print('truncated_new')
# print(np.mean(rand_radius_trunc_new))
# print(np.std(rand_radius_trunc_new))

In [93]:
for n_file in range(existing_files,existing_files+files):
    outputs = np.zeros((points, 29))
    if case == 'case_a':
        rand_points = np.random.uniform(low=lower_lim, high=upper_lim, size=(points, len(var_ranges)))
    if case == 'case_b':
        rand_points = stats.truncnorm((lower_lim-mean_val)/std_val, (upper_lim-mean_val)/std_val, loc=mean_val, scale=std_val).rvs((points, len(var_ranges)))
    if case == 'case_c':
        ### Incredibly complicated!! https://stackoverflow.com/questions/8870982/how-do-i-get-a-lognormal-distribution-in-python-with-mu-and-sigma/36714419#36714419
        rand_points = np.random.uniform(low=lower_lim, high=upper_lim, size=(points, len(var_ranges)))
        rand_radius = np.random.lognormal(mean = -18.165, sigma = 1.01, size = int(points*1.4))
        # rand_radius = stats.lognorm.rvs(math.sqrt(math.log(1 + ((std_val[1])/(mean_val[1]))**2)), 0, (mean_val[1])/math.sqrt(1 + ((std_val[1])/(mean_val[1]))**2),  int(points*1.4))
        rand_points[:,1] = rand_radius[np.where((rand_radius >= lower_lim[1]) & (rand_radius < upper_lim[1]))][:points]

    for i in range(points):
        outputs[i,0:15] = rand_points[i,:]
        results = wu(rand_points[i,:],f_z,f_vis,f_cg,f_rho)
        outputs[i,15:] = results

    np.save('../Results/model_eval/'+case+'/wu_eval_'+str(n_file), outputs)
